In [1]:
import requests
import csv
from datetime import datetime

In [ ]:
TOKEN_ID = {
    "DAI": "0x6b175474e89094c44da98b954eedeac495271d0f",
    "USDT": "0xdac17f958d2ee523a2206206994597c13d831ec7",
    "USDC": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
    "WETH": "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
    "WBTC": "0x2260fac5e5542a773aa44fbcfedf7c193bc2c599",
}
CURRENT_TIME = 1722482066
LAST_UPDATED_TIME = 1659286703
CSV_HEADER_KEYS = ["timestamp", "time", "totalDeposit", "depositRate", "stableBorrowRate", "variableBorrowRate", "utilizationRate", "price", "userCount", "depositers", "borrowers"]

In [ ]:
data = []
for token in TOKEN_ID.keys():
    print("Fetching", token, "data")
    last_timestamp = LAST_UPDATED_TIME
    while True:
        query = '''{
            token(id: \"''' + TOKEN_ID[token] + '''\") {
                _market {
                hourlySnapshots(
                    first: 1000
                    where: {timestamp_gt: \"''' + str(last_timestamp) + '''\", timestamp_lte: \"''' + str(CURRENT_TIME) + '''\"}
                    orderBy: timestamp
                    orderDirection: asc
                ) {
                    rates {
                        rate
                        type
                        side
                    }
                    market {
                        cumulativeUniqueBorrowers
                        cumulativeUniqueDepositors
                        cumulativeUniqueUsers
                    }
                    totalBorrowBalanceUSD
                    totalDepositBalanceUSD
                    inputTokenPriceUSD
                    timestamp
                }
            }
        }
        }'''
        response = requests.post('https://gateway-arbitrum.network.thegraph.com/api/47fe3220cace15c6d958a2b9b3154b74/subgraphs/id/C2zniPn45RnLDGzVeGZCx2Sw3GXrbc9gL4ZfL8B8Em2j'
                                    '',
                                    json={'query': query})
        if response.status_code != 200:
            print("Problem reading data start at timestamp", last_timestamp, ":", response.status_code, response.json())
            continue
        hourlySnapshots = response.json()["data"]["token"]["_market"]["hourlySnapshots"]
        if len(hourlySnapshots) == 0:
            break
        for snapshot in hourlySnapshots:
            item = {}
            item["price"] = snapshot["inputTokenPriceUSD"]
            item["timestamp"] = snapshot["timestamp"]
            item["totalDeposit"] = snapshot["totalDepositBalanceUSD"]
            item["depositRate"] = list(filter(lambda rate: rate["side"] == "LENDER", snapshot["rates"]))[0]["rate"]
            item["stableBorrowRate"] = list(filter(lambda rate: rate["side"] == "BORROWER" and rate["type"] == "STABLE", snapshot["rates"]))[0]["rate"]
            item["variableBorrowRate"] = list(filter(lambda rate: rate["side"] == "BORROWER" and rate["type"] == "VARIABLE", snapshot["rates"]))[0]["rate"]
            item["utilizationRate"] = float(snapshot["totalBorrowBalanceUSD"]) / float(snapshot["totalDepositBalanceUSD"])
            item["time"] = datetime.fromtimestamp(int(item["timestamp"])).isoformat()
            item["userCount"] = snapshot["market"]["cumulativeUniqueUsers"]
            item["depositers"] = snapshot["market"]["cumulativeUniqueDepositors"]
            item["borrowers"] = snapshot["market"]["cumulativeUniqueBorrowers"]
            data.append(item)
        last_timestamp = hourlySnapshots[-1]["timestamp"]
    with open(token + '.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, CSV_HEADER_KEYS)
        dict_writer.writeheader()
        dict_writer.writerows(data)

In [ ]:
!wc * -l

In [ ]:
from google.colab import files # type: ignore
files.download('WETH.csv')
files.download('WBTC.csv')
files.download('DAI.csv')
files.download('USDC.csv')
files.download('USDT.csv')